In [2]:
import pandas as pd
import matplotlib.pylab as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D 
import numpy as np
%matplotlib inline

# PATIENTS

Table source: CareVue and Metavision ICU databases.

Table purpose: Contains all charted data for all patients.

Number of rows: 46,520

Links to:

ADMISSIONS on SUBJECT_ID
ICUSTAYS on SUBJECT_ID

In [3]:
patients =  pd.read_csv("Desktop/cap/PATIENTS.csv")

In [10]:
patients.head(4)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0


In [14]:
patients.EXPIRE_FLAG.value_counts()

0    30761
1    15759
Name: EXPIRE_FLAG, dtype: int64

In [15]:
patients.GENDER.value_counts()

M    26121
F    20399
Name: GENDER, dtype: int64

In [18]:
table = pd.pivot_table(patients, index=['GENDER'], columns=['EXPIRE_FLAG'], values=['ROW_ID'], aggfunc=len, fill_value=0)
table

ROW_ID      
EXPIRE_FLAG      0     1
GENDER                  
F            13164  7235
M            17597  8524

In [21]:
table2 = table.div( table.iloc[:,0], axis=0 )
table2

ROW_ID          
EXPIRE_FLAG      0         1
GENDER                      
F              1.0  0.549605
M              1.0  0.484401

55% Female and 48% male  patients died

In [23]:
r = 7235 + 13164 + 17597 + 8524
table*100/r

ROW_ID           
EXPIRE_FLAG          0          1
GENDER                           
F            28.297506  15.552451
M            37.826741  18.323302

## DOB
DOB is the date of birth of the given patient. Patients who are older than 89 years old at any time in the database have had their date of birth shifted to obscure their age and comply with HIPAA. The shift process was as follows: the patient’s age at their first admission was determined. The date of birth was then set to exactly 300 years before their first admission

DOB has been shifted for patients older than 89. The median age for the patients whose date of birth was shifted is 91.4.

### Note : Let's use 91.4 to sub for teh age of patients over 89.

In [9]:
admission =  pd.read_csv("Downloads/admissions.csv")
patientAdmitted = pd.merge(patients, admission, how='inner', on='SUBJECT_ID')

In [10]:
patientAdmitted.head()

,ROW_ID_x,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,ROW_ID_y,HADM_ID,...,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,321,116935,...,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-12-17 11:10:00,2149-12-17 21:35:00,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,0,1
1,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,322,149546,...,Medicare,ENGL,CATHOLIC,DIVORCED,WHITE,2155-02-03 17:43:00,2155-02-03 21:26:00,GI BLEED/ CHEST PAIN,0,1
2,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,323,158975,...,Medicare,ENGL,CATHOLIC,DIVORCED,WHITE,NaN,NaN,GI BLEEDING\COLONOSCOPY,0,1
3,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1,324,124271,...,Self Pay,HAIT,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2188-11-12 06:56:00,2188-11-12 10:10:00,PNEUMONIA;R/O TB,1,1
4,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0,325,117937,...,Private,NaN,OTHER,NaN,UNKNOWN/NOT SPECIFIED,2110-07-27 05:00:00,2110-07-27 07:15:00,INTRACRANIAL HEAD BLEED,0,1


In [13]:
del patientAdmitted['ROW_ID_x']
del patientAdmitted['ROW_ID_y']

In [15]:
patientAdmitted['DIAGNOSIS'].value_counts()

NEWBORN                                                                                             7823
PNEUMONIA                                                                                           1566
SEPSIS                                                                                              1184
CONGESTIVE HEART FAILURE                                                                             928
CORONARY ARTERY DISEASE                                                                              840
CHEST PAIN                                                                                           778
INTRACRANIAL HEMORRHAGE                                                                              713
ALTERED MENTAL STATUS                                                                                712
GASTROINTESTINAL BLEED                                                                               686
CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /S

In [7]:
patients['AGE'] = ((patients.to_datetime(admission['INTIME']) - pd.to_datetime(df['DOB']))) / np.timedelta64(1, 's') / 3600 / 24

AttributeError: 'DataFrame' object has no attribute 'to_datetime'

Detailed Description

SUBJECT_ID

SUBJECT_ID is a unique identifier which specifies an individual patient. SUBJECT_ID is a candidate key for the table, so is unique for each row. Information that is consistent for the lifetime of a patient is stored in this table.

GENDER

GENDER is the genotypical sex of the patient.

DOB

DOB is the date of birth of the given patient. Patients who are older than 89 years old at any time in the database have had their date of birth shifted to obscure their age and comply with HIPAA. The shift process was as follows: the patient’s age at their first admission was determined. The date of birth was then set to exactly 300 years before their first admission.
DOD, DOD_HOSP, DOD_SSN

DOD is the date of death for the given patient. DOD_HOSP is the date of death as recorded in the hospital database. DOD_SSN is the date of death from the social security database. Note that DOD merged together DOD_HOSP and DOD_SSN, giving priority to DOD_HOSP if both were recorded.

EXPIRE_FLAG

EXPIRE_FLAG is a binary flag which indicates whether the patient died, i.e. whether DOD is null or not. These deaths include both deaths within the hospital (DOD_HOSP) and deaths identified by matching the patient to the social security master death index (DOD_SSN).